In [3]:
from common_nba_api_functions import rate_limited_request
from common_db_functions import get_players_from_db
players = get_players_from_db()
player_ids = [player.player_id for player in players]


In [4]:
season = "2023-2024"
games_by_players = []
for player_id in player_ids:
    rate_limited_request(2)  # Sensible delay
    try:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
        df = gamelog.get_data_frames()[0]
        if df.empty:
            print(f"No games found for player {player_id} in season {season}.")
            continue
        print(df.head())
    except KeyError as ke:
        print(f"KeyError for player {player_id}: {ke}")
        response = gamelog.get_json()
        print("API Response:", response) 
    except Exception as e:
        print(f"Failed for player {player_id}: {str(e)}")

KeyError for player 1630552: 'resultSet'


NameError: name 'gamelog' is not defined

In [5]:
import pandas as pd
from nba_api.stats.endpoints import playergamelog

from common_nba_api_functions import get_gamelog_from_years, get_season_start_and_end


def rate_limited_request(interval):
    import time
    time.sleep(interval)

season = "2023-2024"
season_start_and_end = get_season_start_and_end(season)
date_from = '2023-01-01'  # Start date of the date range
date_to = '2023-01-31'
player_ids = ["201939", "202691", "203507"]  # Example player IDs
player_id = player_ids[0]
gamelog = playergamelog.PlayerGameLog(player_id=player_id, date_from_nullable=date_from, date_to_nullable=date_to)

# Convert the result to a DataFrame
df = gamelog.get_data_frames()[0]

if not df.empty:
    print(df)
else:
    print("No games found for the specified date range.")

2023-24 season's first game: 2023-10-05
The 2023-24 season's last game: 2024-05-10


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
import json
import pandas as pd
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.http import NBAStatsHTTP

def rate_limited_request(interval):
    import time
    time.sleep(interval)

season = "2023-2024"
date_from = "2023-10-05"
date_to = "2024-04-10"
player_ids = ["201939", "202691", "203507"]  # Example player IDs
player_id = player_ids[0]

try:
    gamelog = playergamelog.PlayerGameLog(player_id=player_id, date_from_nullable=date_from, date_to_nullable=date_to)
    nba_stats_http = NBAStatsHTTP()
    response = nba_stats_http.send_api_request(
        endpoint=gamelog.endpoint,
        parameters=gamelog.parameters
    )
    if response.text:
        print("Raw API Response:", response.text)  # Inspect raw response
        df = gamelog.get_data_frames()[0]
        if not df.empty:
            print(df)
        else:
            print("No games found for the specified date range.")
    else:
        print("Empty response received from the API.")
except json.JSONDecodeError as jde:
    print(f"JSON decoding failed: {str(jde)}")
    print("Request sent:", nba_stats_http.last_url, nba_stats_http.last_request_headers)
except Exception as e:
    print(f"An error occurred: {str(e)}")

JSON decoding failed: Expecting value: line 1 column 1 (char 0)


NameError: name 'nba_stats_http' is not defined